###NOTES TO MYSELF:
+ Using a Generator and categorical cross entropy with 1 hot encoding

+ I'm using 2 inputs here

+ I'm also using bidirectional ENCODER

+ using simple GRU decoder

+ No image embedding

+ max sequence length is 12

+ using rmsprop optimizer

-- Check validation

In [1]:
import numpy as np
from numpy import array
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import string
import os
from PIL import Image
import glob
from pickle import dump, load
from time import time
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import LSTM, Embedding, TimeDistributed, Dense, RepeatVector,\
                         Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization, GRU, Masking, Lambda, Concatenate, Average, Reshape
from keras.optimizers import Adam, RMSprop
from keras.layers.wrappers import Bidirectional
from keras.layers.merge import add
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras import Input, layers
from keras import optimizers
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import load_model
import json
import csv

from keras.optimizers import SGD
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
# tf.test.gpu_device_name()


#Wesam
# SEED = 10
#IMAGE_EMBEDDING_VAL_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/validation/'
# IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/Colab_Notebooks/DL_data/one_sample_cnn/'
# filepath = '/content/drive/My Drive/Colab_Notebooks/DL_data/model-ep{epoch:03d}-loss{loss:.3f}.h5'
#CAPTION_FILE_NAME = '/content/drive/My Drive/Colab_Notebooks/DL_data/image_to_caption.csv'


#on my Colab
ALL_CAPTIONS_FILE = '/content/drive/My Drive/dldata/all_captions.txt'
COMPLETE_STORIES_FILE = '/content/drive/My Drive/dldata/complete_stories_all_splits.json'
IMAGE_EMBEDDING_DIR = '/content/drive/My Drive/CNNFeatureVectors/'
GLOVE_EMBEDDING_FILE_NAME = '/content/drive/My Drive/dldata/glove.6B.300d.txt'


# #For my GCP:
# ALL_CAPTIONS_FILE = 'all_captions.txt'
# COMPLETE_STORIES_FILE = 'complete_stories_all_splits.json'
# IMAGE_EMBEDDING_DIR = 'CNNFeatureVectors/'
# GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'


#Vinuta
SEED = 10
IMAGE_EMBEDDING_DIM = 2048
#IMAGE_EMBEDDING_DIR = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/'
NUM_IMAGE_EMBEDDING_CHUNKS = 10
#GLOVE_EMBEDDING_FILE_NAME = 'glove.6B.300d.txt'
MAX_SEQUENCE_LENGTH = 12
WORD_EMBEDDING_DIM = 300
#CAPTION_FILE_NAME = '/Users/vinutahegde/Documents/USC/SEM3/DL/project/ws/image_to_caption.csv'
SENTENCE_EMBEDDING_DIM = 512
BATCH_SIZE = 64

Using TensorFlow backend.


In [2]:
#commented for GCP
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#PreProcess Captions / Stories

Either call this function or simply load preprocessed from a file



In [0]:
#to check if story ids are repeated or unique
# story_list = list()
# for key in list(all_captions_dict.keys()):
#   story_list += list(all_captions_dict[key].keys())
# from collections import Counter
# print(len(story_dict))
# d =  Counter(story_dict)
# res = [k for k, v in d.items() if v > 1]
# print(len(res)) ## gave zero .. so story ids are -in fact- unique


#image_embd =  getImageEmbedding(IMAGE_EMBEDDING_DIR)

def get_existing_stories(image_embeddings):
  #load all_captions file
  with open(ALL_CAPTIONS_FILE) as json_file:
    all_captions_dict = json.load(json_file)

  #Create a story dict (no album ids (already checked that story ids are unique))
  story_dict = {}
  for key in list(all_captions_dict.keys()):
    story_dict.update(all_captions_dict[key])


  # Create a Story dict where all images are available in the image_embeddings
  existing_stories = {}
  c=0
  for key in list(story_dict.keys()):
    lists = story_dict[key]
    images = [item[0] for item in lists]
    #captions = ['startseq ' + item[1] + ' endseq' for item in lists]
    captions = [item[1] for item in lists]
    if all(img in list(image_embeddings.keys()) for img in images):
      existing_stories[key] = [images,captions]
      c+=1
      
  print("Number of Stories Found: ")
  print(c)

  # Saving the complete existing story dict in a file
  with open(COMPLETE_STORIES_FILE, 'w') as fp:
      json.dump(existing_stories, fp)
  return existing_stories

#Vocab

In [0]:
 def vocab_fun(existing_stories_dict):
  index_to_word = {}
  word_to_index = {}
  max_seq_len=0
  all_words = {}
  all_words['startseq'] = 1
  all_words['endseq'] = 1
  cap_list = list()
  for story_id, lists in existing_stories_dict.items():
    for cap in lists[1]:
      if(len(cap.split())>max_seq_len):
        max_seq_len = len(cap.split())
      for word in cap.split():
        all_words[word] = 1
  all_vocab=[w for w in all_words]
  index = 1
  for word in all_vocab:
      word_to_index[word] = index
      index_to_word[index] = word
      index += 1



  #MAX_SEQUENCE_LENGTH = #max_seq_len + 1
  return (all_vocab, word_to_index, index_to_word)

In [0]:
# #to deciede sentence length:

# cap_lengths=[]
# for key, lists in existing_stories.items():
#     cap_list=lists[1]
#     for x in cap_list:
#       for c in x.split():
#         cap_lengths.append(len(c))
# print(np.mean(cap_lengths)+ 2 * np.std(cap_lengths))

#Preprocess images

In [0]:
def Merge(dict1, dict2): 
    res = {**dict1, **dict2} 
    return res 

def getImageEmbedding(path):
    image_embedding = {}
    for i in range(NUM_IMAGE_EMBEDDING_CHUNKS):
         file_name = path + 'cnn_group'+str(i+1)+'.json'
         with open(file_name) as json_file:
#    with open(file_name) as json_file:
            print(file_name)
            json_data = json.load(json_file)
            json_data = json.loads(json_data)
            image_embedding = Merge(image_embedding, json_data) 
            #image_embedding = json_data 
    return image_embedding

#Load Stories (captions with corresponding Image ids)
##Dict items as follows (per story)
[ [img_id1, img_id2, img_id3, img_id4, img_id5] , [cap1, cap2, cap3, cap4, cap5] ]

In [0]:
def get_existing_stories_from_file():
  with open(COMPLETE_STORIES_FILE, 'r') as fp:
      existing_stories = json.load(fp)
  return existing_stories

#Use Prev to get captions / stories and images and pre_process them

In [8]:
#for training
image_embd =  getImageEmbedding(IMAGE_EMBEDDING_DIR)
print(len(image_embd))


/content/drive/My Drive/CNNFeatureVectors/cnn_group1.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group2.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group3.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group4.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group5.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group6.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group7.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group8.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group9.json
/content/drive/My Drive/CNNFeatureVectors/cnn_group10.json
58197


In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
#get existing_stories (either load from file or using a function , preferably load from file) -- uncomment one of the following 2 lines
existing_stories = get_existing_stories_from_file()
#existing_stories = get_existing_stories(image_embd) #Number of Stories Found: 35565

all_vocab, wordtoix, ixtoword=vocab_fun(existing_stories)
print('Max Seq Len: %d' %MAX_SEQUENCE_LENGTH)
vocab_size = len(all_vocab) + 1
print('Vocabulary Size: %d' % vocab_size) #26571

Max Seq Len: 12
Vocabulary Size: 26572


#Word Embedding Matrix

In [11]:
#get matrix embedding for glove
embeddings_index = {} # empty dictionary
f = open(GLOVE_EMBEDDING_FILE_NAME, encoding="utf-8")

for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))


# Get 300-dim dense vector for each of the 10000 words in out vocabulary
embedding_matrix = np.zeros((vocab_size, WORD_EMBEDDING_DIM))
for word, i in wordtoix.items():
    #if i < max_words:
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in the embedding index will be all zeros
        embedding_matrix[i] = embedding_vector

Found 400000 word vectors.


In [0]:
# #save embedding matrix:
# np.save('/content/drive/My Drive/Colab_Notebooks/DL_data/embedding_matrix.npy', embedding_matrix) 

In [0]:
# #save dicts needed during inference:
# with open('/content/drive/My Drive/Colab_Notebooks/DL_data/ixtoword.json', 'w') as fp:
#     json.dump(ixtoword, fp)

# with open('/content/drive/My Drive/Colab_Notebooks/DL_data/wordtoix.json', 'w') as fp:
#     json.dump(wordtoix, fp)

#Input and output for the model
###X1, X2, y

In [0]:

import tensorflow as tf
import tensorflow_hub as hub
def all_data(stories_dict, image_embd, wordtoix, max_length, num_of_stories):
  X1, X2, X3, y = list(), list(), list(), list()
  #to generate X1,X2,X3 and y. ////append-> have a list of image embeds , have a list for curr caption, have sentences embed for previous sentences, next word
  #for each story:
  for key, lists in stories_dict.items():
    #break after retreiving num_of_stories
    if num_of_stories <= 0:
      break
    num_of_stories -= 1
    
    img_list=lists[0]
    img_list_embed=[image_embd[img_id] for img_id in img_list]#[imgtoix[img_id] for img_id in img_list]
    # prev_list=lists[1].copy()
    # prev_list.pop()
    # prev_list.insert(0,'')
    in_cap_list = np.zeros((5,max_length))
    out_cap_list = np.zeros((5,max_length))
    for c in range(5):
      #prev_embeddings = [wordtoix[word] for word in prev_list[c].split() if word in wordtoix]
      #prev_list[c] = pad_sequences([prev_embeddings], maxlen=max_length)[0]

      cap=lists[1][c]
      cap_in = 'startseq ' + cap
      cap_out = cap + ' endseq'
      seq_in = [wordtoix[word] for word in cap_in.split() if word in wordtoix]
      seq_in = pad_sequences([seq_in], maxlen=max_length, padding='post', truncating='post')[0]
      seq_out = [wordtoix[word] for word in cap_out.split() if word in wordtoix]
      seq_out = pad_sequences([seq_out], maxlen=max_length, padding='post', truncating='post')[0]
      if len(cap_out.split()) > max_length:
        seq_out[-1]=wordtoix['endseq']
      in_cap_list[c] = np.array(seq_in)
      out_cap_list[c] = np.array(seq_out)
    X1.append(img_list_embed)
    X2.append(in_cap_list)
    #X3.append(prev_list)
    y.append(np.expand_dims(np.concatenate(out_cap_list).ravel(), -1))##out_cap_list

  return (array(X1), array(X2),  array(y))#array(X3),




In [0]:
#X1,X2,y = all_data(existing_stories, image_embd, wordtoix, MAX_SEQUENCE_LENGTH, 2)#35565

# print("X")
# print(X2)
# print("Y")
# print(y)
# print(np.shape(X2))
# print(np.shape(y))

In [0]:
# print("X")
# print(np.shape(y))

In [0]:
#X1,X2,X3,y = all_data(existing_stories, image_embd, wordtoix, MAX_SEQUENCE_LENGTH, 2)

# print("X")
# print(X2)
# print("Y")
# print(y)

In [0]:
# print(np.shape(y))


#Generator

In [0]:
import keras
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, X1, X2, y, batch_size=32, shuffle=True, vocab_size = 5000):
        'Initialization'
        self.X1=X1
        self.X2=X2
        self.y=y
        self.batch_size = batch_size
        self.list_IDs = list(range(np.shape(X1)[0]))
        self.shuffle = shuffle
        self.vocab_size = vocab_size
        self.on_epoch_end()
        self.max_len = MAX_SEQUENCE_LENGTH

    def someFunction(self,list_IDs_temp):
        return self.__data_generation(list_IDs_temp)
    
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        list_IDs_temp = [self.list_IDs[k] for k in indexes]
        X1_batch,X2_batch, y_batch = self.__data_generation(list_IDs_temp)
        return [X1_batch,X2_batch], y_batch

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' 
        X1_batch = np.empty((self.batch_size, 5, IMAGE_EMBEDDING_DIM))
        X2_batch = np.empty((self.batch_size, 5, self.max_len))
        y_batch = np.empty((self.batch_size, self.max_len*5, 1))

        for i, ID in enumerate(list_IDs_temp):
          X1_batch[i] = self.X1[ID]
          X2_batch[i] = self.X2[ID]
          y_batch[i] = self.y[ID]#np.expand_dims(self.y[ID].copy(), -1)# self.y[ID].copy()


        return X1_batch, X2_batch, y_batch#keras.utils.to_categorical(y_batch, num_classes=self.vocab_size)

In [0]:
# print(X1.shape[0])

In [0]:
# training_generator = DataGenerator(X1,X2,y, vocab_size=vocab_size)

# train, train_Y = training_generator.__getitem__(62)
# train_X1, train_X2 = train
# print(train_X1.shape)
# print(train_X2.shape)
# print(train_Y.shape)

#Building Model

In [0]:
# # Things I should try:
# # 1- bidirectional
# # 2- return sequences and merge the two layers
# # 3- must get good results on validation (the model must be generalizable)
# # 4- modify hyper parameters (hidden layers, batch size, learning rate) (with larger batch size we can use larger learning rate)


from keras.regularizers import l2, l1
from keras import backend as K

def build_model():
###slice = Lambda(lambda x: x[:, i])(input)
    #Image Encoder
    img_input_whole = Input(shape=(5,IMAGE_EMBEDDING_DIM))
    img_input_compressed = TimeDistributed(Dense(640))(img_input_whole)
    img_encoder = GRU(640, recurrent_dropout=0, return_sequences=True , activity_regularizer=l2(0.000))(img_input_compressed)
    img1_enc = Lambda(lambda x: x[:, 0, :])(img_encoder)
    img2_enc = Lambda(lambda x: x[:, 1, :])(img_encoder)
    img3_enc = Lambda(lambda x: x[:, 2, :])(img_encoder)
    img4_enc = Lambda(lambda x: x[:, 3, :])(img_encoder)
    img5_enc = Lambda(lambda x: x[:, 4, :])(img_encoder)
    
    #Embed each:
    Word_Embedder = Embedding(vocab_size, WORD_EMBEDDING_DIM, mask_zero=True, weights=[embedding_matrix], trainable=False)

    #define previouse sentences encoder: #512
    prev_encoder = Bidirectional(GRU(1, recurrent_dropout=0, dropout=0.0 , return_sequences=False, activity_regularizer=l2(0.000)),  merge_mode='ave')

    #Decoder: #512
    decoder = GRU(641,recurrent_dropout=0, dropout=0.0 ,return_sequences=True ,activity_regularizer=l2(0.000))
    decoder2 = GRU(641,recurrent_dropout=0, dropout=0.0 ,return_sequences=True ,activity_regularizer=l2(0.000))
    #Current captions
    captions_input = Input(shape=(5,MAX_SEQUENCE_LENGTH))

    #split
    cap1_in = Lambda(lambda x: x[:, 0, :])(captions_input)
    cap2_in = Lambda(lambda x: x[:, 1, :])(captions_input)
    cap3_in = Lambda(lambda x: x[:, 2, :])(captions_input)
    cap4_in = Lambda(lambda x: x[:, 3, :])(captions_input)
    cap5_in = Lambda(lambda x: x[:, 4, :])(captions_input)
    
    #Embed each:
    cap1_emb = Word_Embedder(cap1_in)
    cap2_emb = Word_Embedder(cap2_in)
    cap3_emb = Word_Embedder(cap3_in)
    cap4_emb = Word_Embedder(cap4_in)
    cap5_emb = Word_Embedder(cap5_in)

    # Concat
    #hidden_1 = concatenate([sent1_enc,img1_enc])

    #Decode
    paddings = tf.constant([[0,0],[0,1]])
    padded_1_image = tf.pad(img1_enc, paddings, mode='CONSTANT',constant_values=1)
    cap1_dec = decoder2(decoder(cap1_emb, initial_state = padded_1_image ))
    cap1_dec = TimeDistributed(Dense(500))(cap1_dec)

    prev_1 = prev_encoder(cap1_dec)
    hidden_2 = concatenate([prev_1,img2_enc])
    cap2_dec = decoder2(decoder(cap2_emb , initial_state = hidden_2))
    cap2_dec = TimeDistributed(Dense(500))(cap2_dec)

    prev_2 = prev_encoder(cap2_dec)
    print(prev_2.shape)
    hidden_3 = concatenate([prev_2,img3_enc])
    cap3_dec = decoder2(decoder(cap3_emb , initial_state = hidden_3))
    cap3_dec = TimeDistributed(Dense(500))(cap3_dec)

    prev_3 = prev_encoder(cap3_dec)
    hidden_4 = concatenate([prev_3,img4_enc])
    cap4_dec = decoder2(decoder(cap4_emb , initial_state = hidden_4))
    cap4_dec = TimeDistributed(Dense(500))(cap4_dec)

    prev_4 = prev_encoder(cap4_dec)
    hidden_5 = concatenate([prev_4,img5_enc])
    cap5_dec = decoder2(decoder(cap5_emb , initial_state = hidden_5))
    cap5_dec = TimeDistributed(Dense(500))(cap4_dec)

    decoder_out = concatenate([cap1_dec, cap2_dec, cap3_dec, cap4_dec, cap5_dec], axis=-2)

    decoder_dense = Dense(512, activation=None, kernel_regularizer=l2(0.000))(decoder_out)
    outputs = Dense(vocab_size, activation='softmax')(decoder_dense) ##was softmax /// used linear because it's recommended with the custom loss: https://github.com/tensorflow/tensorflow/issues/17150
    model = Model(inputs=[img_input_whole, captions_input], outputs=outputs)

    model.summary()
    #decoder_target = tf.placeholder(dtype='int32', shape=(None, None))
    #model.compile(loss=sparse_cross_entropy, optimizer='rmsprop', target_tensors=[decoder_target])#'adam') #, target_tensors=[decoder_target]
    #opt = optimizers.adam(lr = 0.001, beta_1 = 0.9, beta_2 = 0.999, clipnorm = 1.0, clipvalue = 0.5) #0.25 gradeint clipping internet says it's good for LSTM language models
    model.compile(loss='sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['sparse_categorical_accuracy']) #tf.losses.sparse_softmax_cross_entropy #'categorical_crossentropy'

    return model

#Training

In [18]:
model=build_model()

(?, 1)
Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 5, 12)        0                                            
__________________________________________________________________________________________________
lambda_16 (Lambda)              (None, 12)           0           input_4[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 12, 300)      7971600     lambda_16[0][0]                  
                                                                 lambda_17[0][0]                  
                                                                 lambda_18[0][0]                  
                                                                 lambda_19[0][0]     

#Or load saved model

In [0]:
#model = load_model('/content/drive/My Drive/Colab_Notebooks/DL_data/models/final_model_basic_100_stories_checkpoint-ep033-loss3.835.h5')

#Prep for training

In [0]:
X1,X2,y = all_data(existing_stories, image_embd, wordtoix, MAX_SEQUENCE_LENGTH, 1000)#35565
#for GCP
# filepath_checkpoint = "final_model_all_stories-ep{epoch:03d}-loss{loss:.3f}.h5"
# filepath_model = 'final_model_all_stories_finale.h5'

# # #for Colab
# filepath_checkpoint = "/content/drive/My Drive/Colab_Notebooks/DL_data/models/Suggested_10_-ep{epoch:03d}-loss{loss:.3f}.h5"
# filepath_model = '/content/drive/My Drive/Colab_Notebooks/DL_data/models/Suggested_10_epoch1.h5'




In [0]:
split = int(0.9*np.shape(X1)[0])
training_generator = DataGenerator(X1[:split,:,:],X2[:split,:,:],y[:split,:], vocab_size=vocab_size, batch_size=BATCH_SIZE)
validation_generator = DataGenerator(X1[split:,:,:],X2[split:,:,:],y[split:,:], vocab_size=vocab_size, batch_size=BATCH_SIZE)

In [0]:
# from sklearn.utils import class_weight
# con_y = y.
#weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)


#Training

In [20]:
################### Increase batch size!  ##20:53
################### LIMIT VOCAB to 5000, all other things will be UNK
#checkpoint = ModelCheckpoint(filepath_checkpoint, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
#history = model.fit(training_generator, epochs=5, verbose=1, shuffle=True, max_queue_size=5, workers=5, use_multiprocessing=True )#class_weight=weights #callbacks=[checkpoint] 
history = model.fit([X1,X2],y, epochs=20, verbose=1,batch_size = BATCH_SIZE, shuffle=True, max_queue_size=5, workers=10, use_multiprocessing=True, validation_split=0.1 ) 
# model.save('/content/drive/model_epoch10.h5')



Train on 900 samples, validate on 100 samples
Epoch 1/20
900/900 [==============================] - 19s 21ms/step - loss: 8.2323 - sparse_categorical_accuracy: 0.0564 - val_loss: 6.3923 - val_sparse_categorical_accuracy: 0.0299
Epoch 2/20
900/900 [==============================] - 7s 8ms/step - loss: 5.9869 - sparse_categorical_accuracy: 0.0970 - val_loss: 6.1256 - val_sparse_categorical_accuracy: 0.1252
Epoch 3/20
900/900 [==============================] - 7s 8ms/step - loss: 5.7825 - sparse_categorical_accuracy: 0.1123 - val_loss: 6.2253 - val_sparse_categorical_accuracy: 0.0779
Epoch 4/20
900/900 [==============================] - 7s 8ms/step - loss: 5.6076 - sparse_categorical_accuracy: 0.1146 - val_loss: 5.9472 - val_sparse_categorical_accuracy: 0.0924
Epoch 5/20
900/900 [==============================] - 7s 8ms/step - loss: 5.4348 - sparse_categorical_accuracy: 0.1195 - val_loss: 5.6213 - val_sparse_categorical_accuracy: 0.1520
Epoch 6/20
900/900 [==============================

In [0]:
# model.save('/content/drive/My Drive/dldata/epoch20.h5')
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['loss'])
# plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [21]:
history = model.fit([X1,X2],y, epochs=50, verbose=1, batch_size = 16, shuffle=True, max_queue_size=5, workers=10, use_multiprocessing=True, validation_split=0.1 ) 
# model.save('/content/drive/My Drive/dldata/epoch270.h5')
# model.save_weights('/content/drive/My Drive/dldata/model_epoch50_wt.h5')

Train on 900 samples, validate on 100 samples
Epoch 1/50
900/900 [==============================] - 22s 25ms/step - loss: 3.4801 - sparse_categorical_accuracy: 0.2618 - val_loss: 5.5751 - val_sparse_categorical_accuracy: 0.1881
Epoch 2/50
900/900 [==============================] - 22s 25ms/step - loss: 3.2742 - sparse_categorical_accuracy: 0.2938 - val_loss: 5.6972 - val_sparse_categorical_accuracy: 0.1783
Epoch 3/50
900/900 [==============================] - 22s 25ms/step - loss: 3.0428 - sparse_categorical_accuracy: 0.3267 - val_loss: 5.8550 - val_sparse_categorical_accuracy: 0.1671
Epoch 4/50
900/900 [==============================] - 23s 25ms/step - loss: 2.7686 - sparse_categorical_accuracy: 0.3749 - val_loss: 5.8750 - val_sparse_categorical_accuracy: 0.1692
Epoch 5/50
900/900 [==============================] - 23s 25ms/step - loss: 2.4831 - sparse_categorical_accuracy: 0.4272 - val_loss: 6.0335 - val_sparse_categorical_accuracy: 0.1777
Epoch 6/50
900/900 [========================

In [0]:
model.save('/content/drive/My Drive/dldata/pretty_ok_final.h5')
model.save_weights('/content/drive/My Drive/dldata/pretty_ok_final_wt.h5')

#Plot loss curve


In [0]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['loss'])
# plt.plot(history.history['sparse_categorical_accuracy'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [0]:
x1 = X1[0].copy()
x2 = X2[0].copy()
y_ = np.concatenate(y[0].copy()).ravel()
tmp=''
for w in y_:
  if w in ixtoword:
    if w == 'endseq': continue
    tmp += ' ' + ixtoword[w]
print(tmp)

yhat = model.predict([x1[np.newaxis,...],x2[np.newaxis,...]], verbose=0)
print("yhat")
print(np.shape(yhat))

 my trip to location was amazing . endseq i saw some colorful people . endseq i even made some knew friends . endseq i visited a beautiful pagoda . endseq then i saw a ordinary bicycle . endseq
yhat
(1, 100, 26572)


In [0]:
yhat_s = yhat[0,:,:]
# out_x=''
# for j in range(MAX_SEQUENCE_LENGTH*5): 
#     yhat_s = yhat[0,j,:]
#     cur_index_tmp = sample(yhat_s)
#     out_x += ' '+ixtoword[cur_index_tmp]
# print(out_x)

print(yhat_s.shape)
print(np.argmax(yhat_s, axis=-1))
tmp=''
for w in np.argmax(yhat_s, axis=-1):
  if w in ixtoword:
    if w == 2: continue #endseq
    tmp += ' ' + ixtoword[w]
print(tmp)

(100, 26572)
[169 214   5   6   7   8   9   2   2   2   2   2   2   2   2   2   2   2
   2   2  10 122  12  13  14   9   2   2   2   2   2   2   2   2   2   2
   2   2   2   2  45 251  16  12  17  18   9   2   2   2   2   2   2   2
   2   2   2   2   2   2  31   7  20  21  22   9   2   2   2   2   2   2
   2   2   2   2   2   2   2   2 424   7  11  20  24  25   9   9   9   9
   9   9   9   9   9   9   9   9   9   9]
 today first to location was amazing . i am some colorful people . and love made some knew friends . the was a beautiful pagoda . how was saw a ordinary bicycle . . . . . . . . . . . . . .


In [0]:
def get_range_data(stories_dict, image_embd, wordtoix, max_length, start, end):
  X1, X2, X3, y = list(), list(), list(), list()
  #to generate X1,X2,X3 and y. ////append-> have a list of image embeds , have a list for curr caption, have sentences embed for previous sentences, next word
  #for each story:

  story_keys = list(stories_dict.keys())[start:end]
  # for key, lists in stories_dict.items():
  for key in story_keys:
    lists = stories_dict[key]
    img_list=lists[0]
    img_list_embed=[image_embd[img_id] for img_id in img_list]#[imgtoix[img_id] for img_id in img_list]

    in_cap_list = np.zeros((5,max_length))
    out_cap_list = np.zeros((5,max_length))
    for c in range(5):
      cap=lists[1][c]
      cap_in = 'startseq ' + cap
      cap_out = cap + ' endseq'
      seq_in = [wordtoix[word] for word in cap_in.split() if word in wordtoix]
      seq_in = pad_sequences([seq_in], maxlen=max_length, padding='post', truncating='post')[0]
      seq_out = [wordtoix[word] for word in cap_out.split() if word in wordtoix]
      seq_out = pad_sequences([seq_out], maxlen=max_length, padding='post', truncating='post')[0]
      if len(cap_out.split()) > max_length:
        seq_out[-1]=wordtoix['endseq']
      in_cap_list[c] = np.array(seq_in)
      out_cap_list[c] = np.array(seq_out)
    X1.append(img_list_embed)
    X2.append(in_cap_list)
    #X3.append(prev_list)
    y.append(np.expand_dims(np.concatenate(out_cap_list).ravel(), -1))##out_cap_list

  return (array(X1), array(X2),  array(y))#array(X3),

In [0]:
X1,X2,y = get_range_data(existing_stories, image_embd, wordtoix, MAX_SEQUENCE_LENGTH, 2000, 2050)

#TRY validation:

In [0]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

import numpy as np
from scipy.ndimage.interpolation import shift
#X1: 1x5x2048 --images
#X2: 1x5x20  --captions
index_v = 6
from random import choices
# items = range(1, 999)
# new_items = choices(items, k = 20)
for i in range(50):
  index_v = i
  x1_v = X1[index_v].copy()
  x2_v = X2[index_v].copy()

  x2_v_in = np.zeros_like(x2_v)
  x2_v_out = np.zeros_like(x2_v)
  for i in range(5):
    x2_v_in[i][0] = wordtoix['startseq']
    for j in range(MAX_SEQUENCE_LENGTH): 
      yhat = model.predict([x1_v[np.newaxis,...],x2_v_in[np.newaxis,...]], verbose=0)
      yhat_s = yhat[0,(i*MAX_SEQUENCE_LENGTH)+j,:]
      cur_index_tmp = np.argmax(yhat_s)
      #cur_index_tmp = np.random.choice(len(yhat_s), p=yhat_s)
      #cur_index_tmp = sample(yhat_s)
      x2_v_out[i][j] = cur_index_tmp
      if cur_index_tmp == 0 or cur_index_tmp == 2: break ##reached endseq
      if j+1<MAX_SEQUENCE_LENGTH:
        x2_v_in[i][j+1] = cur_index_tmp
      
      

  caps=''
  here = np.concatenate(x2_v_out).ravel()
  for w in here:
    if w in ixtoword:
      if ixtoword[w] == 'startseq' or ixtoword[w] =='endseq': continue
      caps += " " + ixtoword[w]
  # print("Captions:")
  print(caps)

  # print('Original X2')
  or_f = X2[index_v].copy()
  or_f = np.concatenate(or_f).ravel()

  o=''
  for w in or_f:
    if w in ixtoword:
      if ixtoword[w] == 'startseq' or ixtoword[w] =='endseq': continue
      o += " "+ ixtoword[w]
  # print("Original") #X3[1]
  print("-------Original-----"+o)
# the family went to the beach to go fishing . they had a lot of fun activities . they had a lot of fun activities . they had a great time . they had a great time .


 what we need to know before we swim . outside the sights are nice like a large white ferris wheel the streets were welcome to new visitors . the ponds were a beautiful sight to behold . the stay here was full of natural beauty and wonderful wonderful wonderful
-------Original----- a couple went to the beach together . their friend was a third wheel . they had a photographer there to take photos of them . they brought a raft to go out in the water on their friend remained alone under the shade .
 what we need to know before we swim . outside the sights are nice like a large white ferris wheel the streets were welcome to new visitors . the ponds were a beautiful sight to behold . the stay here was full of natural beauty and wonderful wonderful wonderful
-------Original----- we 're here . this little boat survived bringing all of how long do i have to stay this way . i wonderful day for a beach shoot . i love my job what do you think of this pose ? i do n't i could be here all day gettin

In [0]:
print(x2_v_out)
print('---------')
print(x2_v_in)
print(y[0])

[[ 3.  4.  5.  6.  7.  8.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [10. 11. 12. 13. 14.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [10. 15. 16. 12. 17. 18.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [10. 19. 20. 21. 22.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [23. 10. 11. 20. 24. 25.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]
---------
[[ 1.  3.  4.  5.  6.  7.  8.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. 10. 11. 12. 13. 14.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. 10. 15. 16. 12. 17. 18.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. 10. 19. 20. 21. 22.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]
 [ 1. 23. 10. 11. 20. 24. 25.  9.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.]]
[ 3.  4.  5.  6.  7.  8.  9.  2.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 10. 11. 12. 13. 14.  9.  2.  0.  0.  0.  0.  0.  0.  0. 

In [0]:
#save weights:
model.save_weights('/content/drive/My Drive/Colab_Notebooks/DL_data/models/final_model_all_weights_sure.h5')
model.save('/content/drive/My Drive/Colab_Notebooks/DL_data/models/final_model_basic_all_stories_finale_pad_sure.h5')
##model2.load_weights()

In [0]:
# filepath_model_load = '/content/drive/My Drive/Colab_Notebooks/DL_data/models/final_model_basic_100_stories_finale.h5'
#sen3_layer
# model_loaded = load_model(filepath_model_load, custom_objects={'sparse_cross_entropy': tf.nn.softmax_cross_entropy_with_logits})
names = [weight.name for layer in model.layers for weight in layer.weights]
weights = model.get_weights()

for name, weight in zip(names, weights):
    print(name, weight)